<a href="https://colab.research.google.com/github/disha222/Books-Machine-Learning-and-data-Science/blob/main/Basic_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install -q pyspark
!pip install -q handyspark


     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 67.5 MB/s 


In [2]:
pip install handyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from typing import Counter
from pyspark import SparkContext
from pyspark.sql import SparkSession,Window,Row
from pyspark.sql import functions as functions
from pyspark.sql.functions import col,isnan,when,count
from pyspark.sql.functions import *
from pyspark.sql .types import *
#------------------------------------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
from handyspark import *
#-------------------------------------------------------------------------------------------------------------
from pyspark.ml.feature import VectorAssembler
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Building a spark app/session
spark = SparkSession.builder.appName("cementSpark").getOrCreate()

# single cluster information
spark

In [7]:
# Reading the data
df = spark.read.csv('concrete.csv',header=True,inferSchema="true")

#Shape of the dataset
print('Shape of the dataset :',(df.count(),len(df.columns)))

#Displaying top n=10 rows
df.show(n=10)

Shape of the dataset : (1030, 9)
+------+-----+-----+-----+------------+---------+-------+---+--------+
|cement| slag|  ash|water|superplastic|coarseagg|fineagg|age|strength|
+------+-----+-----+-----+------------+---------+-------+---+--------+
| 141.3|212.0|  0.0|203.5|         0.0|    971.8|  748.5| 28|   29.89|
| 168.9| 42.2|124.3|158.3|        10.8|   1080.8|  796.2| 14|   23.51|
| 250.0|  0.0| 95.7|187.4|         5.5|    956.9|  861.2| 28|   29.22|
| 266.0|114.0|  0.0|228.0|         0.0|    932.0|  670.0| 28|   45.85|
| 154.8|183.4|  0.0|193.3|         9.1|   1047.4|  696.7| 28|   18.29|
| 255.0|  0.0|  0.0|192.0|         0.0|    889.8|  945.0| 90|   21.86|
| 166.8|250.2|  0.0|203.5|         0.0|    975.6|  692.6|  7|   15.75|
| 251.4|  0.0|118.3|188.5|         6.4|   1028.4|  757.7| 56|   36.64|
| 296.0|  0.0|  0.0|192.0|         0.0|   1085.0|  765.0| 28|   21.65|
| 155.0|184.0|143.0|194.0|         9.0|    880.0|  699.0| 28|   28.99|
+------+-----+-----+-----+------------+-----

In [9]:
# Getting description of dataset,similar to pandas df.describe()
df.describe().show()

+-------+------------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+
|summary|            cement|             slag|               ash|             water|     superplastic|        coarseagg|          fineagg|               age|          strength|
+-------+------------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+
|  count|              1030|             1030|              1030|              1030|             1030|             1030|             1030|              1030|              1030|
|   mean| 281.1678640776698|73.89582524271844|54.188349514563086|181.56728155339798|6.204660194174756| 972.918932038835|773.5804854368932|45.662135922330094| 35.81796116504854|
| stddev|104.50636449481534|86.27934174810582|63.997004152687644|21.354218565032475| 5.97384139248552|77.7539539667

In [10]:
# Checking for dataframe schema,similar to pandas df.info()
df.printSchema()

root
 |-- cement: double (nullable = true)
 |-- slag: double (nullable = true)
 |-- ash: double (nullable = true)
 |-- water: double (nullable = true)
 |-- superplastic: double (nullable = true)
 |-- coarseagg: double (nullable = true)
 |-- fineagg: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- strength: double (nullable = true)



In [11]:
# Display the values of a particular column
df.select('strength').show()

+--------+
|strength|
+--------+
|   29.89|
|   23.51|
|   29.22|
|   45.85|
|   18.29|
|   21.86|
|   15.75|
|   36.64|
|   21.65|
|   28.99|
|   36.35|
|    6.94|
|   27.92|
|   26.26|
|   23.89|
|   49.97|
|   30.08|
|   44.86|
|   15.04|
|    25.1|
+--------+
only showing top 20 rows



In [12]:
# Display the values of a multiple columns
df.select('age','strength').show()

+---+--------+
|age|strength|
+---+--------+
| 28|   29.89|
| 14|   23.51|
| 28|   29.22|
| 28|   45.85|
| 28|   18.29|
| 90|   21.86|
|  7|   15.75|
| 56|   36.64|
| 28|   21.65|
| 28|   28.99|
| 28|   36.35|
|  3|    6.94|
| 14|   27.92|
|  7|   26.26|
| 28|   23.89|
|100|   49.97|
| 28|   30.08|
| 28|   44.86|
|  3|   15.04|
| 28|    25.1|
+---+--------+
only showing top 20 rows



In [13]:
# Checking the type of dataframe
print(type(df))

# Checking the type of column data
print(type(df.select('strength')))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [14]:
#Displaying datatypes of columns
df.dtypes

[('cement', 'double'),
 ('slag', 'double'),
 ('ash', 'double'),
 ('water', 'double'),
 ('superplastic', 'double'),
 ('coarseagg', 'double'),
 ('fineagg', 'double'),
 ('age', 'int'),
 ('strength', 'double')]

In [15]:
# getting the list of row objects
row_list = df.collect()

#Slicing the python list
part1 = row_list[:5]
part2 = row_list[5:]

#Converting the slices to PySpark Dataframes
slice1 =spark.createDataFrame(part1)
slice2 =spark.createDataFrame(part2)

#printing the first slice
print('First DataFrame')
slice1.show()

#printing the second slice
print('Second DataFrame')
slice2.show()



First DataFrame
+------+-----+-----+-----+------------+---------+-------+---+--------+
|cement| slag|  ash|water|superplastic|coarseagg|fineagg|age|strength|
+------+-----+-----+-----+------------+---------+-------+---+--------+
| 141.3|212.0|  0.0|203.5|         0.0|    971.8|  748.5| 28|   29.89|
| 168.9| 42.2|124.3|158.3|        10.8|   1080.8|  796.2| 14|   23.51|
| 250.0|  0.0| 95.7|187.4|         5.5|    956.9|  861.2| 28|   29.22|
| 266.0|114.0|  0.0|228.0|         0.0|    932.0|  670.0| 28|   45.85|
| 154.8|183.4|  0.0|193.3|         9.1|   1047.4|  696.7| 28|   18.29|
+------+-----+-----+-----+------------+---------+-------+---+--------+

Second DataFrame
+------+-----+-----+-----+------------+---------+-------+---+--------+
|cement| slag|  ash|water|superplastic|coarseagg|fineagg|age|strength|
+------+-----+-----+-----+------------+---------+-------+---+--------+
| 255.0|  0.0|  0.0|192.0|         0.0|    889.8|  945.0| 90|   21.86|
| 166.8|250.2|  0.0|203.5|         0.0|    

In [17]:
# adding a new column to the dataset
df1 = df.withColumn("Cement_new",df['strength']*2)

# displaying the new dataframe
df1.show()

+------+-----+-----+-----+------------+---------+-------+---+--------+----------+
|cement| slag|  ash|water|superplastic|coarseagg|fineagg|age|strength|Cement_new|
+------+-----+-----+-----+------------+---------+-------+---+--------+----------+
| 141.3|212.0|  0.0|203.5|         0.0|    971.8|  748.5| 28|   29.89|     59.78|
| 168.9| 42.2|124.3|158.3|        10.8|   1080.8|  796.2| 14|   23.51|     47.02|
| 250.0|  0.0| 95.7|187.4|         5.5|    956.9|  861.2| 28|   29.22|     58.44|
| 266.0|114.0|  0.0|228.0|         0.0|    932.0|  670.0| 28|   45.85|      91.7|
| 154.8|183.4|  0.0|193.3|         9.1|   1047.4|  696.7| 28|   18.29|     36.58|
| 255.0|  0.0|  0.0|192.0|         0.0|    889.8|  945.0| 90|   21.86|     43.72|
| 166.8|250.2|  0.0|203.5|         0.0|    975.6|  692.6|  7|   15.75|      31.5|
| 251.4|  0.0|118.3|188.5|         6.4|   1028.4|  757.7| 56|   36.64|     73.28|
| 296.0|  0.0|  0.0|192.0|         0.0|   1085.0|  765.0| 28|   21.65|      43.3|
| 155.0|184.0|14

In [18]:
# Droping the newly added column
df2 = df1.drop('Cement_new')
# Displaying the new DataFrame
df2.show()

+------+-----+-----+-----+------------+---------+-------+---+--------+
|cement| slag|  ash|water|superplastic|coarseagg|fineagg|age|strength|
+------+-----+-----+-----+------------+---------+-------+---+--------+
| 141.3|212.0|  0.0|203.5|         0.0|    971.8|  748.5| 28|   29.89|
| 168.9| 42.2|124.3|158.3|        10.8|   1080.8|  796.2| 14|   23.51|
| 250.0|  0.0| 95.7|187.4|         5.5|    956.9|  861.2| 28|   29.22|
| 266.0|114.0|  0.0|228.0|         0.0|    932.0|  670.0| 28|   45.85|
| 154.8|183.4|  0.0|193.3|         9.1|   1047.4|  696.7| 28|   18.29|
| 255.0|  0.0|  0.0|192.0|         0.0|    889.8|  945.0| 90|   21.86|
| 166.8|250.2|  0.0|203.5|         0.0|    975.6|  692.6|  7|   15.75|
| 251.4|  0.0|118.3|188.5|         6.4|   1028.4|  757.7| 56|   36.64|
| 296.0|  0.0|  0.0|192.0|         0.0|   1085.0|  765.0| 28|   21.65|
| 155.0|184.0|143.0|194.0|         9.0|    880.0|  699.0| 28|   28.99|
| 151.8|178.1|138.7|167.5|        18.3|    944.0|  694.6| 28|   36.35|
| 173.

In [19]:
data = df.toPandas()
print('Type of dataframe', type(data))
print('Shape of the dataframe (Pandas Version):', data.shape)
data.head()

Type of dataframe <class 'pandas.core.frame.DataFrame'>
Shape of the dataframe (Pandas Version): (1030, 9)


,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,141.3,212.0,0.0,203.5,0.0,971.8,748.5,28,29.89
1,168.9,42.2,124.3,158.3,10.8,1080.8,796.2,14,23.51
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,28,29.22
3,266.0,114.0,0.0,228.0,0.0,932.0,670.0,28,45.85
4,154.8,183.4,0.0,193.3,9.1,1047.4,696.7,28,18.29


**Data Pre-Processing**

In [22]:
# creating a dataframe to check null value counts
null_df=df.select([count(when(col(c).contains('None')|\

                              col(c).contains('Null')|\

                              (col(c)=='')|\

                              col(c).isNull()|\

                              isnan(c),c

                              )).alias(c)

                              for c in df.columns])



#Displaying the null values counts dataframe

null_df.show()

+------+----+---+-----+------------+---------+-------+---+--------+
|cement|slag|ash|water|superplastic|coarseagg|fineagg|age|strength|
+------+----+---+-----+------------+---------+-------+---+--------+
|     0|   0|  0|    0|           0|        0|      0|  0|       0|
+------+----+---+-----+------------+---------+-------+---+--------+

